In [16]:
import os
import copy
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import models, transforms 
import torch.nn.functional as F
import nibabel as nib
import optuna # Optunaをインポート

from PIL import Image
from tqdm import tqdm # tqdm.notebookからtqdmに変更 (一般的な環境向け)
from sklearn.metrics import precision_recall_curve, auc
import wandb
from datetime import datetime
import itertools




## 学習モデル

In [17]:
# GPUの設定
# CUDA_VISIBLE_DEVICESを設定して、使用するGPUを指定
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # 環境に合わせて設定
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [20]:

# --- グローバル設定 ---
# TODO: ご自身の環境に合わせて設定してください
WANDB_PROJECT_NAME = "vertebrae_axial_bce_a" # 例: W&Bプロジェクト名
TRAIN_CSV_PATH = "/mnt/nfs1/home/yamamoto-hiroto/research/vertebrae/Sakaguchi_file/slice_train/axial/train_labels_axial.csv"        # 例: 訓練用CSVファイルのパス
VAL_CSV_PATH = "/mnt/nfs1/home/yamamoto-hiroto/research/vertebrae/Sakaguchi_file/slice_val/axial/val_labels_axial.csv"            # 例: 検証用CSVファイルのパス
MODEL_SAVE_DIR = "/mnt/nfs1/home/yamamoto-hiroto/research/vertebrae/Sakaguchi_file/S_model_learning/model_pth"               # 例: モデル保存ディレクトリ
SEED = 42 # 再現性のためのシード値
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# データ拡張パラメータ
AUG_RESIZE_SIZE = (224, 224)
AUG_ROTATION_DEGREES = 15  # 陽性サンプルへの回転角度
AUG_TRANSLATE_PERCENT = (0.1, 0.1)  # 平行移動の割合 (幅・高さに対する最大移動割合のタプル)
AUG_SCALE_RANGE = (0.9, 1.1)        # 拡大縮小の範囲 (例: 0.9倍から1.1倍)
AUG_GAUSSIAN_NOISE_STD = 0.05       # ガウスノイズの標準偏差 (0-1正規化後画像への適用を想定)
AUG_GAUSSIAN_NOISE_PROB = 0.5       # ガウスノイズを適用する確率
AUG_NORM_MEAN = (0.5,)
AUG_NORM_STD = (0.5,)

# 学習制御パラメータ
EARLY_STOPPING_PATIENCE = 5 # 検証損失がこのエポック数改善しなければ早期終了
SCHEDULER_PATIENCE = 5     # ReduceLROnPlateau: 検証損失がこのエポック数改善しなければLR低下
SCHEDULER_FACTOR = 0.1     # ReduceLROnPlateau: 学習率を低下させる係数

# --- カスタム変換: ガウスノイズ追加 ---
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=0.05, p=0.5):
        self.std = std
        self.mean = mean
        self.p = p

    def __call__(self, tensor):
        if torch.rand(1).item() < self.p:
            # テンソルが0-1の範囲にあると仮定し、ノイズ付加後にクリップ
            noisy_tensor = tensor + torch.randn(tensor.size()) * self.std + self.mean
            return torch.clamp(noisy_tensor, 0., 1.) # 0-1の範囲に収める
        return tensor

    def __repr__(self):
        return self.__class__.__name__ + f'(mean={self.mean}, std={self.std}, p={self.p})'

# --- データセットクラス (修正版) ---
class CTDataset(Dataset):
    def __init__(self, csv_path, pos_tf=None, neg_tf=None):
        self.data   = pd.read_csv(csv_path)
        self.pos_tf = pos_tf
        self.neg_tf = neg_tf

    def __len__(self):
        return len(self.data)

    def _load_nifti(self, path):
        try:
            img = nib.load(path).get_fdata()
            if img.ndim == 3:
                img = img[:, :, 0]
            elif img.ndim != 2:
                print(f"Warning: Unexpected image dimension {img.ndim} for {path}. Using empty slice.")
                return Image.fromarray(np.zeros(AUG_RESIZE_SIZE, dtype=np.uint8)).convert("L")

            img_min = 100
            img_max = 2000
            img = np.clip(img, img_min, img_max)
            img = (img - img_min) / (img_max - img_min)
            img = (img * 255).astype(np.uint8)
            return Image.fromarray(img).convert("L")
        except FileNotFoundError:
            print(f"Error: File not found at {path}. Returning a black image.")
            return Image.fromarray(np.zeros(AUG_RESIZE_SIZE, dtype=np.uint8)).convert("L")
        except Exception as e:
            print(f"Error loading NIfTI file {path}: {e}. Returning a black image.")
            return Image.fromarray(np.zeros(AUG_RESIZE_SIZE, dtype=np.uint8)).convert("L")

    def __getitem__(self, idx): # ★★★ このメソッドを以下のように置き換えてください ★★★
        r      = self.data.iloc[idx]
        label  = int(r["Fracture_Label"])
        img    = self._load_nifti(r["FullPath"]) # PIL Image

        transform_to_apply = None
        if label == 1:
            # 陽性ラベルの場合、pos_tf を試みる
            transform_to_apply = self.pos_tf
        else: # label == 0
            # 陰性ラベルの場合、neg_tf を試みる
            transform_to_apply = self.neg_tf

        # もし上記の条件で適切な transform が見つからなかった場合
        # (例: label=0 で self.neg_tf が Noneだった場合、または label=1 で self.pos_tf が Noneだった場合)
        # フォールバックとして self.pos_tf を使用する
        # (検証/テスト時や、訓練時に陰性専用の変換がない場合にこれを想定)
        if transform_to_apply is None:
            transform_to_apply = self.pos_tf

        # 最終的に適用する変換が存在するか確認
        if transform_to_apply:
            img = transform_to_apply(img)
        else:
            # このケースは、self.pos_tf すら None の場合であり、設定エラーの可能性が高い。
            raise ValueError(
                f"No transform could be applied for image at index {idx} (label: {label}). "
                "Ensure CTDataset is initialized with `pos_tf` (and optionally `neg_tf`) "
                "containing appropriate transformations including ToTensor()."
            )
        return img, float(label)
    
# --- モデル定義 (変更なし) ---
class ModifiedResNet(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super(ModifiedResNet, self).__init__()
        self.resnet = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(num_features, 1),
        )

    def forward(self, x):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)
        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)
        x = self.resnet.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.resnet.fc(x)
        return x

# --- データ拡張定義 (修正) ---
def get_transforms(resize_size, rotation_degrees, translate_percent, scale_range,
                   gaussian_std, gaussian_prob, norm_mean, norm_std):
    # 訓練時の陽性サンプル用データ拡張
    pos_train_transforms = transforms.Compose([
        transforms.Resize(resize_size),
        transforms.RandomAffine(degrees=rotation_degrees,
                                translate=translate_percent,
                                scale=scale_range,
                                shear=None), # せん断は使用しない
        transforms.ToTensor(), # PIL Imageをテンソルに変換 (0-1スケール)
        AddGaussianNoise(mean=0., std=gaussian_std, p=gaussian_prob),
        transforms.Normalize(mean=norm_mean, std=norm_std)
    ])

    # 訓練時の陰性サンプル用データ拡張 (拡張なし、基本変換のみ)
    neg_train_transforms = transforms.Compose([
        transforms.Resize(resize_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=norm_mean, std=norm_std)
    ])

    # 検証用データ拡張 (拡張なし、基本変換のみ)
    val_transforms = transforms.Compose([
        transforms.Resize(resize_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=norm_mean, std=norm_std)
    ])
    return pos_train_transforms, neg_train_transforms, val_transforms

# --- WeightedRandomSampler 作成関数 (変更なし) ---
def create_sampler(dataset_df): # データセット全体ではなく、DataFrameを受け取るように変更
    # DataFrameからラベルのリストを直接取得
    # CTDatasetインスタンス化前にラベル情報が必要なため
    labels = dataset_df['Fracture_Label'].values 
    class_counts = np.bincount(labels.astype(int))
    if len(class_counts) < 2:
        print("Warning: Only one class found in the dataset. WeightedRandomSampler might not work as expected.")
        return None

    class_weights = 1. / class_counts
    sample_weights = np.array([class_weights[int(l)] for l in labels])
    sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)
    return sampler

# --- 評価関数 (変更なし) ---
def evaluate_model(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_labels, all_preds_probs = [], []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            probs = torch.sigmoid(outputs).cpu().numpy().flatten()
            all_labels.extend(labels.cpu().numpy().flatten())
            all_preds_probs.extend(probs)

    avg_loss = running_loss / len(val_loader)
    prauc = -1.0
    best_threshold = 0.5
    best_precision = 0.0
    best_recall = 0.0

    if len(all_labels) > 0 and len(all_preds_probs) > 0:
        all_labels_np = np.array(all_labels)
        all_preds_probs_np = np.array(all_preds_probs)
        if len(np.unique(all_labels_np)) > 1:
            precision, recall, thresholds = precision_recall_curve(all_labels_np, all_preds_probs_np)
            prauc = auc(recall, precision)
            f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
            if len(f1_scores) > 0:
                best_idx = np.argmax(f1_scores)
                best_threshold = thresholds[min(best_idx, len(thresholds)-1)] if len(thresholds) > 0 else 0.5
                best_precision = precision[best_idx]
                best_recall = recall[best_idx]
        else:
            print(f"Warning: Only one class present in y_true for PRAUC calculation. PRAUC set to 0.")
            prauc = 0.0
    return avg_loss, prauc, best_threshold, best_precision, best_recall

# --- 訓練関数 (早期終了、スケジューラ修正) ---
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device, run_name, early_stopping_patience):
    best_val_prauc = -1.0
    best_epoch_for_prauc = -1
    
    # 早期終了用
    best_val_loss_for_early_stop = float('inf')
    early_stopping_counter = 0

    for epoch in range(num_epochs):
        model.train()
        running_train_loss = 0.0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", leave=False)
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())

        avg_train_loss = running_train_loss / len(train_loader)
        avg_val_loss, val_prauc, best_thr, _, _ = evaluate_model(model, val_loader, criterion, device)
        
        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val PRAUC: {val_prauc:.4f}, Best Thr: {best_thr:.2f}, LR: {optimizer.param_groups[0]['lr']:.2e}")

        wandb.log({
            "epoch": epoch + 1,
            "train_loss": avg_train_loss,
            "val_loss": avg_val_loss,
            "val_prauc": val_prauc,
            "learning_rate": optimizer.param_groups[0]['lr']
        })

        # PRAUCに基づく最良モデルの保存
        if val_prauc > best_val_prauc:
            best_val_prauc = val_prauc
            best_epoch_for_prauc = epoch + 1
            model_path = os.path.join(MODEL_SAVE_DIR, f"{run_name}_best_prauc_model.pth") # シンプルな名前に
            torch.save(model.state_dict(), model_path)
            print(f"New best PRAUC model saved to {model_path} with PRAUC: {best_val_prauc:.4f} at epoch {best_epoch_for_prauc}")
            # wandb.save(model_path) # W&Bに直接保存も可能

        # 早期終了判定 (検証損失に基づく)
        if avg_val_loss < best_val_loss_for_early_stop:
            best_val_loss_for_early_stop = avg_val_loss
            early_stopping_counter = 0
            # 早期終了用に最良の検証損失モデルを保存する場合 (オプション)
            # model_path_es = os.path.join(MODEL_SAVE_DIR, f"{run_name}_best_valloss_model.pth")
            # torch.save(model.state_dict(), model_path_es)
        else:
            early_stopping_counter += 1
        
        if early_stopping_counter >= early_stopping_patience:
            print(f"Early stopping triggered at epoch {epoch+1} as validation loss did not improve for {early_stopping_patience} epochs.")
            break # 学習ループを抜ける

        # スケジューラ更新 (検証損失に基づく)
        if scheduler:
            scheduler.step(avg_val_loss)

    return best_val_prauc, best_epoch_for_prauc

# --- グリッドサーチ関数 (W&B config更新) ---
def grid_search_main(train_csv_path, val_csv_path):
    num_epochs_list = [20]
    lr_list = [1e-6, 1e-3]
    weight_decay_list = [5e-4, 5e-3]
    dropout_rate_list = [0.2]
    batch_size = 32

    pos_train_tf, neg_train_tf, val_tf = get_transforms(
        resize_size=AUG_RESIZE_SIZE,
        rotation_degrees=AUG_ROTATION_DEGREES,
        translate_percent=AUG_TRANSLATE_PERCENT,
        scale_range=AUG_SCALE_RANGE,
        gaussian_std=AUG_GAUSSIAN_NOISE_STD,
        gaussian_prob=AUG_GAUSSIAN_NOISE_PROB,
        norm_mean=AUG_NORM_MEAN,
        norm_std=AUG_NORM_STD
    )

    val_dataset = CTDataset(csv_path=val_csv_path, pos_tf=val_tf) # pos_tfにval_tfを渡す
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    train_df = pd.read_csv(train_csv_path)

    best_overall_prauc = -1.0
    best_params_overall = {}
    
    param_combinations = list(itertools.product(num_epochs_list, lr_list, weight_decay_list, dropout_rate_list))
    print(f"Starting Grid Search with {len(param_combinations)} combinations...")

    for i, params_combo in enumerate(param_combinations):
        current_num_epochs, lr, weight_decay, dropout = params_combo
        run_name = f"e{current_num_epochs}_lr{lr}_wd{weight_decay}_do{dropout}_run{i+1}"
        
        print(f"\n--- Starting Trial {i+1}/{len(param_combinations)}: {run_name} ---")

        wandb.init(
            project=WANDB_PROJECT_NAME,
            name=run_name,
            config={
                "epochs_max": current_num_epochs,
                "learning_rate": lr,
                "weight_decay": weight_decay,
                "dropout_rate": dropout,
                "batch_size": batch_size,
                "architecture": "ModifiedResNet18",
                "scheduler": "ReduceLROnPlateau",
                "scheduler_patience": SCHEDULER_PATIENCE,
                "scheduler_factor": SCHEDULER_FACTOR,
                "early_stopping_patience": EARLY_STOPPING_PATIENCE,
                "sampler": "WeightedRandomSampler_1to1",
                "aug_resize_size": AUG_RESIZE_SIZE,
                "aug_rotation_degrees": AUG_ROTATION_DEGREES,
                "aug_translate_percent": AUG_TRANSLATE_PERCENT,
                "aug_scale_range": AUG_SCALE_RANGE,
                "aug_gaussian_noise_std": AUG_GAUSSIAN_NOISE_STD,
                "aug_gaussian_noise_prob": AUG_GAUSSIAN_NOISE_PROB,
                "seed": SEED + i # 各試行で異なるシードにする場合
            },
            reinit=True
        )
        
        current_seed = SEED + i
        torch.manual_seed(current_seed)
        np.random.seed(current_seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(current_seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

        model = ModifiedResNet(dropout_rate=dropout).to(DEVICE)
        criterion = nn.BCEWithLogitsLoss().to(DEVICE)
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=SCHEDULER_FACTOR, patience=SCHEDULER_PATIENCE, verbose=True)

        train_dataset_instance = CTDataset(csv_path=train_csv_path, pos_tf=pos_train_tf, neg_tf=neg_train_tf)
        sampler = create_sampler(train_df)
        
        if sampler:
            train_loader = DataLoader(train_dataset_instance, batch_size=batch_size, sampler=sampler, num_workers=4, pin_memory=True)
        else:
            train_loader = DataLoader(train_dataset_instance, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

        current_best_prauc_for_trial, _ = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, current_num_epochs, DEVICE, run_name, EARLY_STOPPING_PATIENCE)

        if current_best_prauc_for_trial > best_overall_prauc:
            best_overall_prauc = current_best_prauc_for_trial
            best_params_overall = wandb.config.as_dict()
            print(f"!!! New Overall Best PRAUC: {best_overall_prauc:.4f} with params: {best_params_overall} !!!")

        wandb.summary["best_trial_prauc"] = current_best_prauc_for_trial
        wandb.finish()

    print(f"\n--- Grid Search Finished ---")
    print(f"Best Overall PRAUC: {best_overall_prauc:.4f}")
    print(f"Best Parameters: {best_params_overall}")

if __name__ == "__main__":
    try:
        wandb.login()
    except Exception as e:
        print(f"Could not login to W&B: {e}.")

    grid_search_main(TRAIN_CSV_PATH, VAL_CSV_PATH)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Starting Grid Search with 4 combinations...

--- Starting Trial 1/4: e20_lr1e-06_wd0.0005_do0.2_run1 ---


epoch,▁
learning_rate,▁
train_loss,▁
val_loss,▁
val_prauc,▁
epoch,1
learning_rate,0.0
train_loss,0.42821
val_loss,0.30685
val_prauc,0.33127


/mnt/nfs1/home/yamamoto-hiroto/research/vertebrae/.venv/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


...

Epoch 13/20 - Train Loss: 0.0592, Val Loss: 1.0411, Val PRAUC: 0.0593, Best Thr: 0.00, LR: 1.00e-03


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Epoch 14/20 - Train Loss: 0.0543, Val Loss: 2.2920, Val PRAUC: 0.0654, Best Thr: 0.91, LR: 1.00e-03
Early stopping triggered at epoch 14 as validation loss did not improve for 5 epochs.


epoch,▁▂▂▃▃▄▄▅▅▆▆▇▇█
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▄▂▂▂▂▁▁▁▂▁▁▁▁
val_loss,█▂▆█▁▄▂█▁▁▇▁▂▄
val_prauc,▃▂▃▅▃▁▇█▆▃▄▆▁▂
best_trial_prauc,0.13855
epoch,14
learning_rate,0.001
train_loss,0.05431
val_loss,2.29198
val_prauc,0.06539



--- Grid Search Finished ---
Best Overall PRAUC: 0.3313
Best Parameters: {'epochs_max': 20, 'learning_rate': 1e-06, 'weight_decay': 0.0005, 'dropout_rate': 0.2, 'batch_size': 32, 'architecture': 'ModifiedResNet18', 'scheduler': 'ReduceLROnPlateau', 'scheduler_patience': 5, 'scheduler_factor': 0.1, 'early_stopping_patience': 5, 'sampler': 'WeightedRandomSampler_1to1', 'aug_resize_size': [224, 224], 'aug_rotation_degrees': 15, 'aug_translate_percent': [0.1, 0.1], 'aug_scale_range': [0.9, 1.1], 'aug_gaussian_noise_std': 0.05, 'aug_gaussian_noise_prob': 0.5, 'seed': 42}
